<a href="https://colab.research.google.com/github/sandroormeno/tensorflow_graphics/blob/master/predecir_transformaci%C3%B3n_en_modelo_3d_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_graphics

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow_graphics.geometry.transformation import quaternion
from tensorflow_graphics.math import vector
from tensorflow_graphics.notebooks import threejs_visualization

import numpy as np
import urllib.request

tf.enable_eager_execution()

In [0]:
archivo = "s.obj"
url = "https://raw.githubusercontent.com/sandroormeno/tensorflow_graphics/master/" + archivo
urllib.request.urlretrieve(url, archivo)
file = open(archivo, "r")

In [0]:
vertices = ()
faces = ()
for line in file: 
    my_linea = line
    x = my_linea.split(" ")
    if x[0]== "v":
         # hay dos espacios entre v y los valores 
        vertice_tem = (float(x[2]), float(x[3]), float(x[4])),
        vertices = vertices + vertice_tem
    if x[0]== "f":
         # hay un espacio entre f y los valores 
        cara_tem = (int(x[1]) -1, int(x[2])-1, int(x[3])-1), # para iniciar en CERO, cara O
        faces = faces + cara_tem
vertices = np.array(vertices).astype(np.float32)
faces = np.array(faces).astype(np.float32)
num_vertices = vertices.shape[0]

In [5]:
# https://www.htmlcsscolor.com/hex/808080
def ver():
  context = threejs_visualization.build_context()
  light1 = context.THREE.PointLight.new_object(0XDB3838)
  light1.position.set(50., 10., 10.)
  light2 = context.THREE.AmbientLight.new_object(0x6AA84F)# 0x808080
  light2.position.set(-50., 10., 10.)
  lights = (light1, light2)
  material = context.THREE.MeshPhongMaterial.new_object( { 'color': 0xCCCCCC, 'dithering': True , } );
  mesh = {'vertices': vertices, 'faces': faces, 'material': material, 'castShadow' : True , 'receiveShadow' : True }
  camera = threejs_visualization.build_perspective_camera(
          field_of_view=30, position=(10.0, 10.0, 10.0))
  width = 200
  height = 200
  _ = threejs_visualization.triangular_mesh_renderer([mesh],
                                                     width=width,
                                                     height=height,
                                                     camera=camera,
                                                   lights=lights)
ver()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# Constructs the model.
model = keras.Sequential()
model.add(layers.Flatten(input_shape=(num_vertices, 3)))
model.add(layers.Dense(64, activation=tf.nn.tanh))
model.add(layers.Dense(64, activation=tf.nn.relu))
model.add(layers.Dense(7))


def pose_estimation_loss(y_true, y_pred):
  # y_true.shape : (batch, 7)
  y_true_q, y_true_t = tf.split(y_true, (4, 3), axis=-1)
  # y_pred.shape : (batch, 7)
  y_pred_q, y_pred_t = tf.split(y_pred, (4, 3), axis=-1)

  # vertices.shape: (num_vertices, 3)
  # corners.shape:(num_vertices, 1, 3)
  corners = tf.expand_dims(vertices, axis=1)

  # transformed_corners.shape: (num_vertices, batch, 3)
  # q and t shapes get pre-pre-padded with 1's following standard broadcast rules.
  transformed_corners = quaternion.rotate(corners, y_pred_q) + y_pred_t

  # recovered_corners.shape: (num_vertices, batch, 3)
  recovered_corners = quaternion.rotate(transformed_corners - y_true_t,
                                        quaternion.inverse(y_true_q))

  # vertex_error.shape: (num_vertices, batch)
  vertex_error = tf.reduce_sum((recovered_corners - corners)**2, axis=-1)

  return tf.reduce_mean(vertex_error)


optimizer = keras.optimizers.Adam()
model.compile(loss=pose_estimation_loss, optimizer=optimizer)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


W0516 19:45:27.152024 140622430836608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/resource_variable_ops.py:642: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 144)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                9280      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 13,895
Trainable params: 13,895
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_training_data(num_samples):
  # random_angles.shape: (num_samples, 3)
  random_angles = np.random.uniform(-np.pi, np.pi,
                                    (num_samples, 3)).astype(np.float32)

  # random_quaternion.shape: (num_samples, 4)
  random_quaternion = quaternion.from_euler(random_angles)

  # random_translation.shape: (num_samples, 3)
  random_translation = np.random.uniform(-2.0, 2.0,
                                         (num_samples, 3)).astype(np.float32)

  # data.shape : (num_samples, num_vertices, 3)
  data = quaternion.rotate(vertices[tf.newaxis, :, :],
                           random_quaternion[:, tf.newaxis, :]
                          ) + random_translation[:, tf.newaxis, :]


  # target.shape : (num_samples, 4+3)
  target = tf.concat((random_quaternion, random_translation), axis=-1)

  return np.array(data), np.array(target)

In [0]:
num_samples = 30000

data, target = generate_training_data(num_samples)


In [0]:
# Callback allowing to display the progression of the training task.
class ProgressTracker(keras.callbacks.Callback):

  def __init__(self, num_epochs, step=5):
    self.num_epochs = num_epochs
    self.current_epoch = 0.
    self.step = step
    self.last_percentage_report = 0

  def on_epoch_end(self, batch, logs={}):
    self.current_epoch += 1.
    training_percentage = int(self.current_epoch * 100.0 / self.num_epochs)
    if training_percentage - self.last_percentage_report >= self.step:
      print('Training ' + str(
          training_percentage) + '% complete. Training loss: ' + str(
              logs.get('loss')) + ' | Validation loss: ' + str(
                  logs.get('val_loss')))
      self.last_percentage_report = training_percentage

In [0]:
reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    verbose=0,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0)

In [11]:
# Everything is now in place to train.
EPOCHS = 100
pt = ProgressTracker(EPOCHS)
history = model.fit(
    data,
    target,
    epochs=EPOCHS,
    validation_split=0.2,
    verbose=0,
    batch_size=32,
    callbacks=[reduce_lr_callback, pt])


Instructions for updating:
Use tf.cast instead.


W0516 19:46:13.350914 140622430836608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Training 5% complete. Training loss: 0.15604736781617007 | Validation loss: 0.16296758550405502
Training 10% complete. Training loss: 0.10007380284120639 | Validation loss: 0.0947694166302681
Training 15% complete. Training loss: 0.07821367359658082 | Validation loss: 0.0766801559527715
Training 20% complete. Training loss: 0.0637396983926495 | Validation loss: 0.0665423223276933
Training 25% complete. Training loss: 0.058841901974131666 | Validation loss: 0.06918076415856679
Training 30% complete. Training loss: 0.054322327146927515 | Validation loss: 0.055002408449848494
Training 35% complete. Training loss: 0.05328541054452459 | Validation loss: 0.053410310342907906
Training 40% complete. Training loss: 0.04438072718059023 | Validation loss: 0.05182333178321521
Training 45% complete. Training loss: 0.03306646849277119 | Validation loss: 0.03648721041282018
Training 50% complete. Training loss: 0.02946516752615571 | Validation loss: 0.03831141869723797
Training 55% complete. Training

In [0]:
# Defines the loss function to be optimized.
def transform_points(target_points, quaternion_variable, translation_variable):
  return quaternion.rotate(target_points,
                           quaternion_variable) + translation_variable

In [0]:
class Viewer(object):

  def __init__(self, my_vertices):
    my_vertices = np.asarray(my_vertices)
    context = threejs_visualization.build_context()
    light1 = context.THREE.PointLight.new_object(0x808080)
    light1.position.set(10., 10., 10.)
    light2 = context.THREE.AmbientLight.new_object(0x808080)
    lights = (light1, light2)

    material = context.THREE.MeshLambertMaterial.new_object({
        'color': 0x3D85C6,
    })

    material_deformed = context.THREE.MeshLambertMaterial.new_object({
        'color': 0xE06666,
    })

    camera = threejs_visualization.build_perspective_camera(
        field_of_view=30, position=(10.0, 10.0, 10.0))

    mesh = {'vertices': vertices, 'faces': faces, 'material': material}
    transformed_mesh = {
        'vertices': my_vertices,
        'faces': faces,
        'material': material_deformed
    }
    geometries = threejs_visualization.triangular_mesh_renderer(
        #[mesh, transformed_mesh],
        [transformed_mesh],
        lights=lights,
        camera=camera,
        width=200,
        height=200)

    self.geometries = geometries

  def update(self, transformed_points):
    self.geometries[1].getAttribute('position').copyArray(
        transformed_points.numpy().ravel().tolist())
    self.geometries[1].getAttribute('position').needsUpdate = True

In [0]:
def get_random_transform():
  # Forms a random translation
  with tf.name_scope('translation_variable'):
    random_translation = tf.Variable(
        np.random.uniform(-2.0, 2.0, (3,)), dtype=tf.float32)

  # Forms a random quaternion
  hi = np.pi
  lo = -hi
  random_angles = np.random.uniform(lo, hi, (3,)).astype(np.float32)
  with tf.name_scope('rotation_variable'):
    random_quaternion = tf.Variable(quaternion.from_euler(random_angles))

  return random_quaternion, random_translation


In [22]:
random_quaternion, random_translation = get_random_transform()

initial_orientation = transform_points(vertices, random_quaternion,
                                       random_translation).numpy()
viewer = Viewer(initial_orientation)
ver()

predicted_transformation = model.predict(initial_orientation[tf.newaxis, :, :])

predicted_inverse_q = quaternion.inverse(predicted_transformation[0, 0:4])
predicted_inverse_t = -predicted_transformation[0, 4:]

predicted_aligned = quaternion.rotate(initial_orientation + predicted_inverse_t,
                                      predicted_inverse_q)

viewer = Viewer(predicted_aligned)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>